# Displaying Data

#### 1. Listing all collections (Tables) within the database

In [30]:
from Connections import list_collections
list_collections()

Collections in 'PRJ382_DB':
- your_collection_name
- ESP32_data


#### 2. Displaying Data in Tabular format

In [31]:
from Connections import show_documents
show_documents("ESP32_data" , 10)

Documents in collection 'ESP32_data':
+--------------------------+----------------------+-----------------+------------------------+-----------------+--------------------+----------------+---------------+--------------------------------------------------+------------------+
| _id                      | timestamp            | device_id       |   solar_irradiance_wm2 |   temperature_c |   humidity_percent |   pressure_hpa |   rainfall_mm | gps                                              |   lte_signal_dbm |
+==========================+======================+=================+========================+=================+====================+================+===============+==================================================+==================+
| 6896e1e1ec8614bff080ebfd | 2025-08-08T10:00:00Z | ESP32-WROOM-001 |                  908.5 |            25.2 |                 52 |         1013   |          2.16 | {'lat': -25.7461, 'lon': 28.1881, 'alt_m': 1340} |              -68 |
+-------------

#### 3. Changing the displayed data into a dataframe for better calculating

In [11]:
from Connections import client, DEFAULT_DB  # import these so you can access MongoDB
import pandas as pd

def show_documents_df(collection_name, limit=None):
    db = client[DEFAULT_DB]
    collection = db[collection_name]
    cursor = collection.find()
    if limit:
        cursor = cursor.limit(limit)
    documents = list(cursor)
    if not documents:
        print(f"No documents found in collection '{collection_name}'.")
        return pd.DataFrame()
    for doc in documents:
        if '_id' in doc:
            doc['_id'] = str(doc['_id'])
    df = pd.DataFrame(documents)
    return df

# Now call your new function
df = show_documents_df("ESP32_data", 50)
df.head()

,_id,timestamp,device_id,solar_irradiance_wm2,temperature_c,humidity_percent,pressure_hpa,rainfall_mm,gps,lte_signal_dbm
0,6896e1e1ec8614bff080ebfd,2025-08-08T10:00:00Z,ESP32-WROOM-001,908.5,25.2,52,1013.0,2.16,"{'lat': -25.7461, 'lon': 28.1881, 'alt_m': 1340}",-68
1,6896e1e1ec8614bff080ebfe,2025-08-08T10:01:00Z,ESP32-WROOM-001,935.6,25.6,59,1011.4,4.26,"{'lat': -25.7461, 'lon': 28.1881, 'alt_m': 1340}",-62
2,6896e1e1ec8614bff080ebff,2025-08-08T10:02:00Z,ESP32-WROOM-001,855.3,27.9,56,1010.8,0.44,"{'lat': -25.7461, 'lon': 28.1881, 'alt_m': 1340}",-63
3,6896e1e1ec8614bff080ec00,2025-08-08T10:03:00Z,ESP32-WROOM-001,842.7,27.9,60,1012.9,0.65,"{'lat': -25.7461, 'lon': 28.1881, 'alt_m': 1340}",-60
4,6896e1e1ec8614bff080ec01,2025-08-08T10:04:00Z,ESP32-WROOM-001,896.8,26.3,57,1011.0,1.29,"{'lat': -25.7461, 'lon': 28.1881, 'alt_m': 1340}",-62


# Cleaning The Table

### 1. Handling Missing Data

In [22]:
# Fill missing numeric data with median values only (ignore non-numeric columns)

numeric_cols = df.select_dtypes(include='number').columns
df_filled = df.copy()
df_filled[numeric_cols] = df_filled[numeric_cols].fillna(df_filled[numeric_cols].median())

print("Filled missing numeric values with median for columns:")
print(numeric_cols)

# Optionally show how many missing remain
print("\nRemaining missing values after fill (should be zero for numeric columns):")
print(df_filled[numeric_cols].isna().sum())

Filled missing numeric values with median for columns:
Index(['solar_irradiance_wm2', 'temperature_c', 'humidity_percent',
       'pressure_hpa', 'rainfall_mm', 'lte_signal_dbm'],
      dtype='object')

Remaining missing values after fill (should be zero for numeric columns):
solar_irradiance_wm2    0
temperature_c           0
humidity_percent        0
pressure_hpa            0
rainfall_mm             0
lte_signal_dbm          0
dtype: int64


### 2. Dealing with Duplicate Values

In [25]:
# Identify columns with unhashable data (like dicts or lists)
unhashable_cols = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, (dict, list))).any()]

print("Columns with unhashable data types that will be excluded from duplicates check:")
print(unhashable_cols)

df_subset = df.drop(columns=unhashable_cols)

num_duplicates = df_subset.duplicated().sum()
print(f"Number of duplicates (excluding unhashable columns): {num_duplicates}")


df_clean = df.loc[~df_subset.duplicated()].copy()

print(f"Rows before dropping duplicates: {len(df)}")
print(f"Rows after dropping duplicates: {len(df_clean)}")

# Replace original df with cleaned one if you want
df = df_clean

Columns with unhashable data types that will be excluded from duplicates check:
['gps']
Number of duplicates (excluding unhashable columns): 0
Rows before dropping duplicates: 50
Rows after dropping duplicates: 50


#### 3. Handling Outliers

In [26]:
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

df = remove_outliers_iqr(df, 'temperature_c')

#### 4. Handling Impossible Values

In [ ]:
#Rainfall can never be a negative number and humidity percentage can never be more than 100 or less than 0
df = df[df['rainfall_mm'] >= 0]
df = df[(df['humidity_percent'] >= 0) & (df['humidity_percent'] <= 100)]

#### 5. Sync MongoDB with DataFrame

In [29]:
from bson import ObjectId

db = client[DEFAULT_DB]
collection = db['your_collection_name']  # replace with your actual collection name

# Count documents before deleting
count_before = collection.count_documents({})

# Convert _id strings back to ObjectId if needed
if '_id' in df.columns:
    df['_id'] = df['_id'].apply(lambda x: ObjectId(x) if isinstance(x, str) else x)

# Delete all existing documents
collection.delete_many({})

# Convert DataFrame to dict records
records = df.to_dict(orient='records')

# Insert cleaned records
insert_result = collection.insert_many(records)

count_after = len(insert_result.inserted_ids)

print(f"Number of documents deleted: {count_before}")
print(f"Number of documents inserted: {count_after}")
print(f"Number of documents changed: {abs(count_after - count_before)}")

Number of documents deleted: 50
Number of documents inserted: 50
Number of documents changed: 0
